In [19]:
import numpy as np
from typing import List, Tuple, Dict
import logging
from concurrent.futures import ThreadPoolExecutor
from queue import Queue
import time
import matplotlib.pyplot as plt
from scipy.stats import norm, gamma
from sklearn.preprocessing import StandardScaler

In [20]:
# Configuração de logging
logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class SimulationConfig:
    """Configuração para simulação Monte Carlo"""
    def __init__(self, n_simulations: int, batch_size: int, n_variables: int,
                 confidence_level: float, optimization_target: float):
        self.n_simulations = n_simulations
        self.batch_size = batch_size
        self.n_variables = n_variables
        self.confidence_level = confidence_level
        self.optimization_target = optimization_target

class Distribution:
    """Classe base para distribuições probabilísticas"""
    def sample(self, size: int) -> np.ndarray:
        raise NotImplementedError

    def params(self) -> dict:
        raise NotImplementedError

class OptimizedNormalDistribution(Distribution):
    """Distribuição normal otimizada"""
    def __init__(self, mu: float, sigma: float, rng: np.random.Generator):
        self.mu = mu
        self.sigma = sigma
        self.rng = rng
        self._cached_samples = None
        self._cache_size = 10000

    def sample(self, size: int) -> np.ndarray:
        if self._cached_samples is None or len(self._cached_samples) < size:
            self._cached_samples = self.rng.normal(
                self.mu, self.sigma, self._cache_size
            )

        result = self._cached_samples[:size]
        self._cached_samples = self._cached_samples[size:]
        return result

    def params(self) -> dict:
        return {'mu': self.mu, 'sigma': self.sigma}

class SimulationResult:
    """Resultados da simulação com métricas"""
    def __init__(self, data: np.ndarray, metrics: Dict[str, float]):
        self.data = data
        self.metrics = metrics
        self.timestamp = time.time()

    def get_confidence_interval(self, confidence_level: float) -> Tuple[float, float]:
        """Calcula intervalo de confiança"""
        z_score = norm.ppf((1 + confidence_level) / 2)
        mean = np.mean(self.data)
        std = np.std(self.data) / np.sqrt(len(self.data))
        return mean - z_score * std, mean + z_score * std

class MonteCarloEngine:
    """Motor de simulação Monte Carlo otimizado"""
    def __init__(self, config: SimulationConfig):
        self.config = config
        self.rng = np.random.default_rng(seed=42)
        self.results_queue = Queue()
        self.distributions: List[Distribution] = []
        self._initialize_distributions()

    def _initialize_distributions(self):
        """Inicializa distribuições para simulação"""
        for _ in range(self.config.n_variables):
            mu = self.rng.normal(0, 1)
            sigma = self.rng.uniform(0.5, 2)
            self.distributions.append(
                OptimizedNormalDistribution(mu, sigma, self.rng)
            )

    def _simulate_batch(self, batch_size: int) -> np.ndarray:
        """Simula um lote de dados"""
        results = np.zeros((batch_size, len(self.distributions)))
        for i, dist in enumerate(self.distributions):
            results[:, i] = dist.sample(batch_size)
        return results

    def _process_batch_results(self, batch_data: np.ndarray) -> Dict[str, float]:
        """Processa resultados do lote"""
        return {
            'mean': float(np.mean(batch_data)),
            'std': float(np.std(batch_data)),
            'var': float(np.var(batch_data)),
            'skew': float(gamma.stats(a=3, moments='s')),  # Exemplo simplificado
            'kurtosis': float(gamma.stats(a=3, moments='k'))  # Exemplo simplificado
        }

    def _optimize_parameters(self, batch_results: List[SimulationResult]) -> Dict[str, float]:
        """Otimiza parâmetros baseado nos resultados"""
        all_data = np.concatenate([r.data for r in batch_results])
        target = self.config.optimization_target

        # Otimização usando gradiente descendente simplificado
        learning_rate = 0.01
        n_iterations = 100
        params = np.ones(all_data.shape[1]) / all_data.shape[1]

        for _ in range(n_iterations):
            predicted = np.sum(params * all_data, axis=1)
            error = predicted - target
            gradient = 2 * np.mean(error[:, np.newaxis] * all_data, axis=0)
            params -= learning_rate * gradient
            params = np.clip(params, 0, 1)
            params /= np.sum(params)

        return {'optimal_weights': params}

    def run_simulation(self) -> List[SimulationResult]:
        """Executa simulação Monte Carlo completa"""
        logger.info("Iniciando simulação Monte Carlo...")
        results = []

        with ThreadPoolExecutor() as executor:
            futures = []
            for i in range(0, self.config.n_simulations, self.config.batch_size):
                batch_size = min(
                    self.config.batch_size,
                    self.config.n_simulations - i
                )
                futures.append(
                    executor.submit(self._simulate_batch, batch_size)
                )

            for future in futures:
                batch_data = future.result()
                metrics = self._process_batch_results(batch_data)
                results.append(SimulationResult(batch_data, metrics))

        logger.info("Simulação concluída. Otimizando parâmetros...")
        optimization_results = self._optimize_parameters(results)
        logger.info(f"Otimização concluída: {optimization_results}")

        return results

class SimulationAnalyzer:
    """Analisador de resultados da simulação"""
    def __init__(self, results: List[SimulationResult]):
        self.results = results
        self.scaler = StandardScaler()

    def compute_aggregate_metrics(self) -> Dict[str, float]:
        """Computa métricas agregadas"""
        all_data = np.concatenate([r.data for r in self.results])
        scaled_data = self.scaler.fit_transform(all_data)

        return {
            'mean': float(np.mean(all_data)),
            'std': float(np.std(all_data)),
            'correlation_matrix': np.corrcoef(all_data.T),
            'principal_components': np.linalg.svd(scaled_data)[1]
        }

    def plot_distributions(self):
        """Plota distribuições resultantes"""
        all_data = np.concatenate([r.data for r in self.results])

        plt.figure(figsize=(15, 10))
        for i in range(all_data.shape[1]):
            plt.subplot(2, 2, i + 1)
            plt.hist(all_data[:, i], bins=50, density=True, alpha=0.7)
            plt.title(f'Distribuição Variável {i+1}')
            plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

    def plot_correlation_heatmap(self):
        """Plota mapa de calor de correlações"""
        all_data = np.concatenate([r.data for r in self.results])
        corr_matrix = np.corrcoef(all_data.T)

        plt.figure(figsize=(10, 8))
        plt.imshow(corr_matrix, cmap='coolwarm', aspect='auto')
        plt.colorbar()
        plt.title('Matriz de Correlação')
        plt.tight_layout()
        plt.show()

def main():
    # Configuração da simulação
    config = SimulationConfig(
        n_simulations=100000,
        batch_size=1000,
        n_variables=4,
        confidence_level=0.95,
        optimization_target=1.0
    )

    # Executa simulação
    engine = MonteCarloEngine(config)
    results = engine.run_simulation()

    # Analisa resultados
    analyzer = SimulationAnalyzer(results)
    metrics = analyzer.compute_aggregate_metrics()

    # Visualiza resultados
    analyzer.plot_distributions()
    analyzer.plot_correlation_heatmap()

    # Imprime métricas
    logger.info("Métricas Finais:")
    for key, value in metrics.items():
        if isinstance(value, np.ndarray):
            logger.info(f"{key}:\n{value}")
        else:
            logger.info(f"{key}: {value}")

if __name__ == "__main__":
    main()

2024-12-24 00:02:42,386 - INFO - Iniciando simulação Monte Carlo...
2024-12-24 00:02:42,429 - INFO - Simulação concluída. Otimizando parâmetros...
2024-12-24 00:02:42,981 - INFO - Otimização concluída: {'optimal_weights': array([0.59760911, 0.35254718, 0.        , 0.04984371])}


MemoryError: Unable to allocate 74.5 GiB for an array with shape (100000, 100000) and data type float64